# Harmonic Search Algorithm
Implementation of algorithm 3 in the paper "A Novel Discrete Global-Best Harmony Search Algorithm for Solving 0-1 Knapsack Problems" by Xiang *et al*, http://dx.doi.org/10.1155/2014/573731

In [11]:
using PyCall
t = pyimport("pymatgen")

PyObject <module 'pymatgen' from '/project/6000101/chinchay/mydocs/venvs/asenv/lib/python3.8/site-packages/pymatgen/__init__.py'>

### Load functions from `HSA.jl`

In [12]:
include("./HSA.jl")

getEnergyLammps (generic function with 1 method)

### Define the system

Parameters you can change/play:

L1: number of impurities of type1  
L2: number of impurities of type2  
Nv: total number of available vacancies to fill out with impurities of type1 or type2  
cutoff: electrostatic interaction cutoff in Angstrom


In [13]:
L1     = 10    #2
L2     = 418   #50
Nv     = 576   #72
cutOff = 10.0 #10.0
file = "testVac/data.lammps_222" ;# update:LiLaZnO structure definition (208 atom's positions and charges in LAMMPS format)

### Energy calculation of the system with no impurities:  `Nv` vacancies not filled

In [14]:
# the following shouldn't be modified:

L      = L1 + L2 # total number of impurities to be inserted into the garnet  
ion1   = 1
ion2   = 2
Ne     = Nv - L # number of vacancies that will be left empty after filling with imprurities  
distSite2Col, neighbors_of, charges, ion1, ion2, removedSites, U, UionAionBlist = initialize(L1, L2, Nv, cutOff, file)

energyBase = getEnergyBase(removedSites, charges, distSite2Col, neighbors_of)
println("Energy of garnet with Nv vacancies not filled: ", energyBase, " eV")

il        = zeros(Int, L);
Nv_list   = collect(1:Nv);
tempListL = zeros(Int,L);

Energy of garnet with Nv vacancies not filled: 6670.083783916868 eV


### Adjust the PAR parameter
If `is_PAR_dynamic` is set `false` then only `par` is used as a constant value at each iteration  
If `is_PAR_dynamic` is set `true`  then a dynamically PAR update will be used at each iteration, going from `parMin` to `parMax`  
Remember, `par` belongs to [0.0, 1.0]

In [15]:
par    = 1.0

is_PAR_dynamic = false # options: false, true
parMin = 0.0  # 0.0
parMax = 0.0 ;# 1.0

### Adjust the HCMR parameter
If `is_HCMR_dynamic` is set `false` then only `hcmr` is used as a constant value at each iteration  
If `is_HCMR_dynamic` is set `true`  then a dynamically HCMR update will be used at each iteration, going from `hcmrMin` to `hcmrMax`  
Remember, `hcmr` belongs to [0.0, 1.0]


In [16]:
hcmr    = 1.0

is_HCMR_dynamic = false
hcmrMin = 0.0 # 0.0
hcmrMax = 0.0 ;# 1.0

### Adjust the memory size value
Here we test with different memory matrix size values, stored in `memory_vals`

In [17]:
# memory_vals = collect( 10:10:10 )
memory_vals = collect( 10:10:100 ) ;# this will define a list of Nmem values: [10, 20, ..., 100]

# if you want to run once only, with Nmem=10 then uncomment:
# memory_vals = collect( 10:10 ) ;# this will define a list of only one Nmem value: [10]

### Adjust the number of iterations
For each run, try increasing `nIterations` to get closer to the ground state energy

In [18]:
nIterations = 100_000 ;# 200_000

### Adjust the number of runs
`nruns` will allow us to repeat the simulation if `nrun` > 1

In [19]:
nruns = 5 ;#5

### Simulation

While running the next cell, the output will have the following format:  

k, Nmem: 1 10  
length(listAvoidBest): 9  
-5451.288272121392  
-5481.51812848723  
[14, 58, 59, 24, 7, 30, 42, 61, 64, 50, 68, 27, 20, 71, 63, 60, 34, 25, 22, 66, 28, 13, 1, 19, 15, 55, 5, 49, 47, 67, 10, 72, 52, 39, 51, 29, 9, 23, 4, 45, 18, 41, 3, 62, 31, 17, 44, 37, 48, 56, 32, 69]  

Meaning:  
The `k`-th simulation uses `Nmem`=10 (matrix memory size = 10)  
Here, -5451.2... and -5481.5... represents the worst and best energy values (eV) reached in this simulation  
[14, 58, ...] (=`il`) is a list of `L` numbers going from `1` to `Nv` representing the sites occupied by ions


In [20]:
# the following shouldn't be modified:

n = length(memory_vals)
yBestMean  = zeros(n)
yBestDev   = zeros(n)
yWorstMean = zeros(n)
yWorstDev  = zeros(n)
yTimeMean  = zeros(n)
yTimeDev   = zeros(n)
# for EACH simulation, we need to use:
bestE_hist  = zeros(nIterations)
worstE_hist = zeros(nIterations)
par_hist    = zeros(nIterations) # used if nruns == 1

bestHarmony = zeros(Int, L)
emptiesDict = Dict{Int,Bool}( site => true for site in 1:Nv )

experiment11!(nruns, L, Nv, Nv_list, il, ion1, ion2, U, UionAionBlist, nIterations, energyBase, memory_vals, yBestMean, yBestDev, yWorstMean, yWorstDev, yTimeMean, yTimeDev, bestE_hist, worstE_hist, par_hist, bestHarmony, emptiesDict, par, is_PAR_dynamic, parMin, parMax, hcmr, is_HCMR_dynamic, hcmrMin, hcmrMax )


# This is for checking with LAMMPS calculations. It's very slow, so first set nIterations = 100 to test.
#u1 = get_il_Ut_plus_Uion(L1, L, il, U, UionAionBlist, ion1, ion2, energyBase)
#u2 = getEnergyLammps(L1, L, il)
#if abs(u1 - u2) > 0.1
#    throw(error())
#    println(u1, " /// ", u2)
## else
##     println(u1, " ... ", u2)
#end

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
k, Nmem: 1 10
length(listAvoidBest): 9
-42615.71667468004
-42656.13444377809
[385, 130, 28, 538, 234, 228, 172, 72, 570, 399, 168, 252, 522, 357, 39, 349, 201, 395, 416, 58, 268, 361, 207, 49, 404, 237, 525, 30, 288, 94, 436, 317, 311, 132, 170, 297, 171, 202, 134, 149, 107, 515, 59, 27, 491, 438, 387, 417, 215, 518, 537, 214, 466, 260, 363, 528, 353, 310, 192, 104, 22, 303, 204, 131, 197, 490, 456, 447, 145, 212, 24, 559, 45, 17, 394, 61, 210, 147, 254, 220, 376, 178, 383, 135, 18, 124, 184, 222, 275, 80, 150, 565, 304, 1, 79, 503, 25, 191, 489, 233, 426, 433, 281, 57, 77, 89, 502, 557, 173, 180, 499, 523, 463, 536, 306, 289, 126, 148, 462, 78, 37, 248, 377, 166, 378, 354, 461, 231, 386, 440, 402, 142, 50, 526, 469, 15, 200, 295, 185, 425, 513, 274, 360, 88, 364, 285, 476, 97, 509, 381, 296, 244, 277, 429, 573, 410, 267, 556, 96, 406, 12, 101, 348, 157, 412, 382, 82, 327, 133, 457, 508, 541, 336, 545, 35, 156, 465, 280, 547, 41, 241, 571, 307,

-42656.69471592105
-42723.69664583418
[375, 214, 464, 126, 300, 336, 131, 49, 223, 383, 42, 220, 196, 89, 177, 429, 18, 349, 317, 152, 563, 206, 217, 508, 232, 254, 253, 201, 455, 319, 381, 235, 87, 306, 204, 261, 367, 442, 105, 413, 522, 243, 35, 102, 115, 288, 489, 388, 31, 422, 558, 245, 346, 473, 251, 91, 193, 36, 330, 406, 181, 250, 560, 434, 500, 525, 170, 426, 104, 463, 357, 564, 207, 57, 416, 472, 191, 433, 142, 203, 175, 512, 39, 226, 448, 529, 97, 378, 533, 1, 485, 441, 231, 312, 365, 462, 514, 518, 382, 299, 144, 110, 190, 438, 504, 493, 432, 351, 83, 225, 275, 523, 571, 94, 397, 331, 23, 553, 278, 476, 46, 155, 161, 183, 554, 267, 501, 298, 436, 404, 562, 80, 552, 132, 371, 93, 164, 240, 65, 301, 74, 17, 260, 468, 565, 494, 499, 151, 160, 96, 390, 116, 400, 156, 84, 446, 71, 427, 148, 20, 341, 276, 376, 219, 377, 538, 4, 90, 28, 145, 403, 281, 130, 273, 410, 385, 262, 124, 419, 575, 399, 327, 452, 221, 335, 470, 239, 418, 149, 325, 409, 154, 64, 297, 445, 316, 69, 443, 73, 

-42612.57957295674
-42762.673308680685
[187, 400, 529, 301, 57, 228, 462, 288, 393, 463, 1, 132, 452, 296, 140, 508, 474, 206, 9, 88, 6, 138, 573, 492, 8, 123, 429, 20, 526, 575, 513, 417, 97, 176, 74, 93, 224, 401, 549, 98, 218, 416, 231, 15, 236, 101, 129, 348, 488, 55, 85, 279, 280, 382, 239, 495, 262, 87, 261, 387, 500, 554, 325, 412, 144, 221, 32, 432, 350, 274, 339, 214, 289, 446, 10, 536, 443, 259, 332, 571, 233, 380, 162, 190, 62, 56, 298, 178, 445, 430, 18, 473, 209, 61, 100, 509, 338, 30, 158, 563, 212, 431, 537, 191, 19, 90, 297, 565, 35, 121, 106, 421, 374, 149, 501, 499, 237, 349, 290, 104, 73, 438, 78, 80, 540, 300, 14, 351, 5, 367, 268, 522, 227, 478, 361, 466, 71, 410, 525, 406, 192, 36, 461, 265, 151, 378, 59, 447, 568, 415, 486, 334, 26, 66, 258, 491, 246, 251, 404, 564, 76, 27, 453, 342, 109, 391, 379, 207, 25, 177, 273, 370, 545, 37, 570, 188, 267, 359, 287, 392, 315, 161, 225, 105, 223, 125, 189, 241, 307, 154, 244, 205, 86, 83, 131, 255, 441, 518, 383, 229, 559, 2

-42573.84062013161
-42682.61386810768
[87, 527, 380, 22, 293, 58, 135, 319, 243, 535, 558, 1, 44, 117, 235, 238, 67, 312, 53, 121, 304, 104, 532, 526, 415, 394, 198, 433, 127, 445, 554, 128, 306, 284, 123, 357, 522, 190, 393, 371, 236, 412, 516, 237, 285, 470, 373, 277, 439, 105, 163, 111, 368, 320, 27, 230, 542, 175, 504, 199, 296, 302, 40, 463, 444, 106, 499, 338, 23, 391, 524, 289, 2, 119, 464, 217, 263, 454, 416, 265, 440, 515, 313, 480, 334, 573, 411, 432, 551, 329, 213, 276, 134, 221, 170, 251, 227, 477, 182, 422, 565, 194, 80, 448, 418, 348, 511, 88, 155, 267, 317, 144, 26, 131, 19, 246, 508, 410, 360, 273, 478, 176, 34, 83, 11, 572, 174, 46, 447, 355, 189, 255, 73, 428, 172, 43, 361, 152, 406, 437, 459, 207, 553, 241, 567, 490, 311, 383, 124, 503, 17, 216, 202, 458, 279, 35, 520, 325, 557, 133, 250, 64, 366, 421, 328, 211, 307, 225, 427, 399, 36, 137, 245, 308, 301, 468, 438, 413, 452, 280, 200, 398, 556, 186, 481, 82, 187, 300, 389, 142, 222, 561, 108, 476, 115, 143, 171, 419,

LoadError: [91mInterruptException:[39m

In [21]:
if nruns > 1
    y1max = 15.0
    y3max = 10.0
    xlabel = "HMS"
    plot2(1.0*memory_vals, yBestMean, yBestDev, yWorstMean, yWorstDev, yTimeMean, yTimeDev, y1max, y3max, xlabel)
else
    plot1("hsa_", nIterations, bestE_hist, worstE_hist, par_hist)
end

In [24]:
length(yBestDev)

10

In [31]:
io = open("yBestMean", "w")
println(io, yBestMean)
close(io)

io = open("yBestDev", "w")
println(io, yBestDev)
close(io)

io = open("yWorstMean", "w")
println(io, yWorstMean)
close(io)

io = open("yWorstDev", "w")
println(io, yWorstDev)
close(io)

io = open("yTimeMean", "w")
println(io, yTimeMean)
close(io)

In [35]:
println(yBestMean)
println(yBestDev)
println(yWorstMean)
println(yWorstDev)
println(yTimeMean)

[-37109.996542212626, -37136.35122163505, -37132.522755780046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[38.673290069628855, 29.476875488385872, 40.489894653608104, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-37036.40397620834, -37018.34442493064, -37007.07197416498, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[17.603591622440256, 7.229503431580945, 14.373225993691793, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[118.30926936339999, 120.3404901614, 118.72285634340001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [30]:
# using Queryverse
# f = open("xx", "w")
# writedlm(f, [yBestMean, yBestDev, yWorstMean, yWorstDev, yTimeMean])
# close(f)
# println(f, yBestMean)
# println(f, yBestDev)
# println(f, yWorstMean)
# println(f, yWorstDev)
# println(f, yTimeMean)
# close(f)

LoadError: [91mtype IOStream has no field write[39m

In [17]:
bestE_hist[10000]

-42595.690880150265

In [19]:
yBestMean

1-element Array{Float64,1}:
 -37155.796740697675